In [ ]:
import numpy as np
from numpy import shape

from datetime import date

import random
import math
import dill
import glob
import gc

# from tensorflow import keras

import matplotlib.pyplot as plt

from random import seed

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import device

# Tensorflow soll auf CPU und nicht auf der GPU laufen
device("cpu:0")
# für GPU:
# tf.device("gpu:0")

In [129]:
%run '../../rna/_functions/functions.py'

### Load images

In [44]:
filepath_data_combined = "../data/combined_data/constructed_images_combined_60_2022_08_25.csv"
filepath_labels = "../data/combined_data/constructed_labels_60_2022_08_25.csv"

X = np.genfromtxt(filepath_data_combined, delimiter = ",")
y = np.array(pd.read_csv(filepath_labels, header=None))

# Factorize into dichotomous variable
Y = pd.factorize(y[:, 0])[0]

### Load Persistences Landscapes

In [8]:
pl_folderpath = "../data/persistence_landscapes/"

In [98]:
# # Eingelesenen Dateien
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

avgPL_bucket01_H0_scaledWithin = load_file(file = glob.glob(pl_folderpath + "PL_H0_scaledWithin*.pkl")[-1])
avgPL_bucket01_H1_scaledWithin = load_file(file = glob.glob(pl_folderpath + "PL_H1_scaledWithin*.pkl")[-1])

avgPL_bucket01_H0_scaledBetween = load_file(file = glob.glob(pl_folderpath + "PL_H0_scaledBetween*.pkl")[-1])
avgPL_bucket01_H1_scaledBetween = load_file(file = glob.glob(pl_folderpath + "PL_H1_scaledBetween*.pkl")[-1])

avgPL_bucket01_H0_unscaled = load_file(file = glob.glob(pl_folderpath + "PL_H0_unscaled*.pkl")[-1])
avgPL_bucket01_H1_unscaled = load_file(file = glob.glob(pl_folderpath + "PL_H1_unscaled*.pkl")[-1])

In [25]:
# Parameter for persistence landscapes
pl_resolution = 200
pl_num_landscapes = 10

### Train-Test-Splits Persistence Landscapes

In [99]:
# Bucket01 scaledBetween
seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H0_train, X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_test = train_test_split(avgPL_bucket01_H0_scaledBetween,
                                                                                                                                        Y,
                                                                                                                                        test_size=0.2)

seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H1_train, X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_test = train_test_split(avgPL_bucket01_H1_scaledBetween,
                                                                                                                                        Y,
                                                                                                                                        test_size=0.2)

# Bucket01 scaledWithin
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H0_train, X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_test = train_test_split(avgPL_bucket01_H0_scaledWithin,
                                                                                                                                    Y,
                                                                                                                                    test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H1_train, X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_test = train_test_split(avgPL_bucket01_H1_scaledWithin,
                                                                                                                                    Y,
                                                                                                                                    test_size=0.2)

# Bucket01 Unscaled
seed(999)
# Train-Test-Split 
X_b01_unscaled_H0_train, X_b01_unscaled_H0_test, Y_b01_unscaled_H0_train, Y_b01_unscaled_H0_test = train_test_split(avgPL_bucket01_H0_unscaled,
                                                                                                                    Y,
                                                                                                                    test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_unscaled_H1_train, X_b01_unscaled_H1_test, Y_b01_unscaled_H1_train, Y_b01_unscaled_H1_test = train_test_split(avgPL_bucket01_H1_unscaled,
                                                                                                                    Y,
                                                                                                                    test_size=0.2)

# Machine Learning Classification

## Support Vector Machine

In [27]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [ ]:
folderpath_results = "../results/"

### Train-Test-Split raw data

In [90]:
# Train-test split with data in vector format
image_vector_size = 6*6

X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X.reshape(len(X), image_vector_size),
                                                                    np.asarray(y),
                                                                    test_size = 0.3,
                                                                    random_state = 2)

##### SVM (raw data)

In [137]:
# Linear - Rawdata
clf = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_raw_train, y_raw_train.ravel())

print("Accuracy SVM on raw data: " + str(clf.score(X_raw_test, y_raw_test))[:10])

evaluation(y_test = y_raw_test,
           y_pred = clf.predict(X_raw_test),
           filename_csv = folderpath_results + "svm_linear_rawdata.csv")

Accuracy SVM on raw data: 1.0
Confusion-Matrix:
 [[10  0]
 [ 0  8]] 
 
True Negative:  10 
True Positive 8 
False Negative 0 
False Positive 0

TPR: 1.0 
TNR:  1.0

Accuracy: 1.0
F1 (micro): 1.0
F1 (macro): 1.0


In [138]:
# RBF - Rawdata
clf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_raw_train, y_raw_train.ravel())

print("Accuracy SVM on raw data: " + str(clf.score(X_raw_test, y_raw_test))[:10])

evaluation(y_test = y_raw_test,
           y_pred = clf.predict(X_raw_test),
           filename_csv = folderpath_results + "svm_rbf_rawdata.csv")

Accuracy SVM on raw data: 1.0
Confusion-Matrix:
 [[10  0]
 [ 0  8]] 
 
True Negative:  10 
True Positive 8 
False Negative 0 
False Positive 0

TPR: 1.0 
TNR:  1.0

Accuracy: 1.0
F1 (micro): 1.0
F1 (macro): 1.0


##### SVM (Persistent Landscapes  - scaledWithin)

In [148]:
# Linear - H0 - scaledWithin
clf_pl_h0 = make_pipeline(StandardScaler(), 
                       SVC(kernel='linear')).fit(X_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_h0.score(X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H0_test,
           y_pred = clf_pl_h0.predict(X_b01_scaledWithin_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.75
Confusion-Matrix:
 [[3 2]
 [1 6]] 
 
True Negative:  3 
True Positive 6 
False Negative 1 
False Positive 2

TPR: 0.8571428571428571 
TNR:  0.6

Accuracy: 0.75
F1 (micro): 0.75
F1 (macro): 0.7333333333333332


In [140]:
# Linear - H1 - scaledWithin
clf_pl = make_pipeline(StandardScaler(), 
                       SVC(kernel='linear')).fit(X_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl.score(X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H1_test,
           y_pred = clf_pl.predict(X_b01_scaledWithin_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.75
Confusion-Matrix:
 [[6 3]
 [0 3]] 
 
True Negative:  6 
True Positive 3 
False Negative 0 
False Positive 3

TPR: 1.0 
TNR:  0.6666666666666666

Accuracy: 0.75
F1 (micro): 0.75
F1 (macro): 0.7333333333333334


In [141]:
# RBF - H0 - scaledWithin
clf_pl_h0 = make_pipeline(StandardScaler(), 
                       SVC(kernel='rbf')).fit(X_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_h0.score(X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H0_test,
           y_pred = clf_pl_h0.predict(X_b01_scaledWithin_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.75
Confusion-Matrix:
 [[3 2]
 [1 6]] 
 
True Negative:  3 
True Positive 6 
False Negative 1 
False Positive 2

TPR: 0.8571428571428571 
TNR:  0.6

Accuracy: 0.75
F1 (micro): 0.75
F1 (macro): 0.7333333333333332


In [142]:
# RBF - H1 - scaledWithin
clf_pl_rbf = make_pipeline(StandardScaler(), 
                       SVC(kernel='rbf')).fit(X_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_rbf.score(X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_scaledWithin_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.83333333
Confusion-Matrix:
 [[8 1]
 [1 2]] 
 
True Negative:  8 
True Positive 2 
False Negative 1 
False Positive 1

TPR: 0.6666666666666666 
TNR:  0.8888888888888888

Accuracy: 0.8333333333333334
F1 (micro): 0.8333333333333334
F1 (macro): 0.7777777777777777


##### SVM (Persistent Landscapes - scaledBetween)

In [143]:
# Linear - H0 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H0_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_scaledBetween.csv")

Accuracy SVM : 0.58333333
Confusion-Matrix:
 [[5 2]
 [3 2]] 
 
True Negative:  5 
True Positive 2 
False Negative 3 
False Positive 2

TPR: 0.4 
TNR:  0.7142857142857143

Accuracy: 0.5833333333333334
F1 (micro): 0.5833333333333334
F1 (macro): 0.5555555555555556


In [144]:
# Linear - H1 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H1_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_scaledBetween.csv")

Accuracy SVM : 0.91666666
Confusion-Matrix:
 [[6 0]
 [1 5]] 
 
True Negative:  6 
True Positive 5 
False Negative 1 
False Positive 0

TPR: 0.8333333333333334 
TNR:  1.0

Accuracy: 0.9166666666666666
F1 (micro): 0.9166666666666666
F1 (macro): 0.916083916083916


In [145]:
# RBF - H0 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H0_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_scaledBetween.csv")

Accuracy SVM : 0.91666666
Confusion-Matrix:
 [[6 1]
 [0 5]] 
 
True Negative:  6 
True Positive 5 
False Negative 0 
False Positive 1

TPR: 1.0 
TNR:  0.8571428571428571

Accuracy: 0.9166666666666666
F1 (micro): 0.9166666666666666
F1 (macro): 0.916083916083916


In [146]:
# RBF - H1 - scaledBetween
clf_pl_rbf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_train)

print("Accuracy SVM : " + str(clf_pl_rbf.score(X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_scaledBetween_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_scaledBetween.csv")

Accuracy SVM : 0.83333333
Confusion-Matrix:
 [[4 2]
 [0 6]] 
 
True Negative:  4 
True Positive 6 
False Negative 0 
False Positive 2

TPR: 1.0 
TNR:  0.6666666666666666

Accuracy: 0.8333333333333334
F1 (micro): 0.8333333333333334
F1 (macro): 0.8285714285714285
